<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Lesson 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)
## _aka_ PREDICTING THE FUTURE!

<img src="https://media.giphy.com/media/l2JJu8U8SoHhQEnoQ/giphy.gif" width=480 height=356>
<br></br>
<br></br>

> "Yesterday's just a memory - tomorrow is never what it's supposed to be." -- Bob Dylan

Wish you could save [Time In A Bottle](https://www.youtube.com/watch?v=AnWWj6xOleY)? With statistics you can do the next best thing - understand how data varies over time (or any sequential order), and use the order/time dimension predictively.

A sequence is just any enumerated collection - order counts, and repetition is allowed. Python lists are a good elemental example - `[1, 2, 2, -1]` is a valid list, and is different from `[1, 2, -1, 2]`. The data structures we tend to use (e.g. NumPy arrays) are often built on this fundamental structure.

A time series is data where you have not just the order but some actual continuous marker for where they lie "in time" - this could be a date, a timestamp, [Unix time](https://en.wikipedia.org/wiki/Unix_time), or something else. All time series are also sequences, and for some techniques you may just consider their order and not "how far apart" the entries are (if you have particularly consistent data collected at regular intervals it may not matter).

## Recurrent Neural Networks

There's plenty more to "traditional" time series, but the latest and greatest technique for sequence data is recurrent neural networks. A recurrence relation in math is an equation that uses recursion to define a sequence - a famous example is the Fibonacci numbers:

$F_n = F_{n-1} + F_{n-2}$

For formal math you also need a base case $F_0=1, F_1=1$, and then the rest builds from there. But for neural networks what we're really talking about are loops:

![Recurrent neural network](https://upload.wikimedia.org/wikipedia/commons/b/b5/Recurrent_neural_network_unfold.svg)

The hidden layers have edges (output) going back to their own input - this loop means that for any time `t` the training is at least partly based on the output from time `t-1`. The entire network is being represented on the left, and you can unfold the network explicitly to see how it behaves at any given `t`.

Different units can have this "loop", but a particularly successful one is the long short-term memory unit (LSTM):

![Long short-term memory unit](https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Long_Short-Term_Memory.svg/1024px-Long_Short-Term_Memory.svg.png)

There's a lot going on here - in a nutshell, the calculus still works out and backpropagation can still be implemented. The advantage (ane namesake) of LSTM is that it can generally put more weight on recent (short-term) events while not completely losing older (long-term) information.

After enough iterations, a typical neural network will start calculating prior gradients that are so small they effectively become zero - this is the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem), and is what RNN with LSTM addresses. Pay special attention to the $c_t$ parameters and how they pass through the unit to get an intuition for how this problem is solved.

So why are these cool? One particularly compelling application is actually not time series but language modeling - language is inherently ordered data (letters/words go one after another, and the order *matters*). [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) is a famous and worth reading blog post on this topic.

For our purposes, let's use TensorFlow and Keras to train RNNs with natural language. Resources:

- https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py
- https://keras.io/layers/recurrent/#lstm
- http://adventuresinmachinelearning.com/keras-lstm-tutorial/

Note that `tensorflow.contrib` [also has an implementation of RNN/LSTM](https://www.tensorflow.org/tutorials/sequences/recurrent).

### RNN/LSTM Sentiment Classification with Keras

In [10]:
'''
#Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
**Notes**
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [11]:
len(x_train)

25000

In [12]:
x_train[0][:50]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447]

In [13]:
# all training observations have variousl lengths lengths
len(x_train[200]), len(x_train[2467])

(160, 321)

In [14]:
# Trim down observations to specific lengths
# if less than 80, pads them with zeros
# if more than 80
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [15]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 97s 4ms/sample - loss: 0.4577 - accuracy: 0.7821 - val_loss: 0.3705 - val_accuracy: 0.8364
Epoch 2/15
25000/25000 [==============================] - 94s 4ms/sample - loss: 0.2941 - accuracy: 0.8808 - val_loss: 0.3778 - val_accuracy: 0.8345
Epoch 3/15
25000/25000 [==============================] - 97s 4ms/sample - loss: 0.2088 - accuracy: 0.9191 - val_loss: 0.4224 - val_accuracy: 0.8366
Epoch 4/15
25000/25000 [==============================] - 97s 4ms/sample - loss: 0.1498 - accuracy: 0.9444 - val_loss: 0.4824 - val_accuracy: 0.8125
Epoch 5/15
25000/25000 [==============================] - 97s 4ms/sample - loss: 0.1073 - accuracy: 0.9610 - val_loss: 0.5902 - val_accuracy: 0.8229
Epoch 6/15
25000/25000 [==============================] - 99s 4ms/sample - loss: 0.0760 - accuracy: 0.9744 - val_loss: 0.7446 - val_accuracy: 0.8141
Epoch 7/15
25000/25000 [========

### LSTM Text generation with Keras

What else can we do with LSTMs? Since we're analyzing the *sequence*, we can do more than classify - we can *generate* text. I'ved pulled some news stories using [newspaper](https://github.com/codelucas/newspaper/).

This example is drawn from the Keras [documentation](https://keras.io/examples/lstm_text_generation/).

In [43]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [44]:
data_files = os.listdir('./articles')

In [45]:
# Read in Data
data = []
for file in data_files:
    if file[-3:] == 'txt':
        with open(f'./articles/{file}', 'r', encoding='utf-8') as f:
            data.append(f.read())

In [46]:
data[50][:100]

'Welcome to week four of the impeachment inquiry. Stay up to date with us — reach out & sign up.\n\nEll'

In [47]:
len(data)

136

In [48]:
# Encode Data as Chars
text = " ".join(data)
chars = list(set(text))
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [50]:
# Create the Sequence Data
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]
sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])

print('sequences:', len(sequences))

sequences: 178374


In [51]:
# Specify x & y

# x is a 3 dimensional matrix
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1
        
    y[i, next_chars[i]] = 1

In [52]:
x.shape, y.shape

((178374, 40, 121), (178374, 121))

In [53]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [54]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [103]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]
            # print("next_char: " + next_char)

            sentence = sentence[1:] + next_char
            # print("sentence[1:]: " + sentence[1:])

            sys.stdout.write(next_char)
            # sys.stdout.flush() # This is bugging, not adding any spaces
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [104]:
on_epoch_end(5, _)


----- Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: "owered onto Trump’s face.

By late Sunda"
owered onto Trump’s face.

By late Sunday said the court in the comment of the conters in thecount the compentthe comment of the compents that the cate the commenta sentions of the contertion of the can compents a said the came the compents that the seasing the compent of the count the country hasa sending the contertion of the confersting the comesthat thereading the president and the country the president ofthe contertion and t
----- diversity: 0.5
----- Generating with seed: "owered onto Trump’s face.

By late Sunda"
owered onto Trump’s face.

By late Sunday and the come the tame in 2019 call a thebe secure white aman be had a fitesectionsaction with the reters to counce tothe counted to the relear collice in and who said thathe was allowed that deart a feet todears the fire expeation with the and the dass as a work for in the searing theTurkish and the c

In [105]:
model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Train on 178374 samples
Epoch 1/5
 23936/178374 [===>..........................] - ETA: 1:09 - loss: 1.8011
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "feed in a manner inconsistent with any p"
feed in a manner inconsistent with any prome that thepromed the comment andthereal thatthe said the residethe reledthe reserve thepresidentof the reasingthecomment of thereporttothe president oftheServicesof thereally and the commentof the has a deficethereside the prome the reason that the prosident and the residethe promeof thereside the president ofthe prome the presidentof the ServicesandServicesand the r
----- diversity: 0.5
----- Generating with seed: "feed in a manner inconsistent with any p"
feed in a manner inconsistent with any promber to the regultfor the fillled to subleeveris apreation of the Persice. He has lead releatof the commans areto in theServices ergenand Trumpthatwe led the part theWashington Post in the last to thesaidthatin to ca

KeyboardInterrupt: 

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [ ]:
get_available_gpus()